# Introuduction:
This is the lab-2 introducing the basic version of a Deep Learning Model implenentation in pytorch.

# Class Inheritence
This section introduces the basic strucure of class inheritance which we will follow in pytorch for inheriting our classes from torch.nn.Modules.

In [ ]:
class parent ():
    def __init__(self):
        print("Constructor of Parent Class")
class child(parent):
    def __init__(self):
        super().__init__()
        print("Constructor of Child Class")

obj = child()


Constructor of Parent Class
Constructor of Child Class


Now Lets define a DenseLayer class

In [5]:
import torch
import torch.nn as nn

In [ ]:

class ourDenseLayer(torch.nn.Module):
    def __init__(self,num_input,num_output):
        super(ourDenseLayer, self).__init__()
        self.w = torch.nn.Parameter(torch.rand(num_input,num_output))
        self.bias = torch.nn.Parameter(torch.rand(num_output))
    def forward(self,x):
        z = torch.matmul(x, self.w) +self.bias
        y = torch.sigmoid(z)
        return y


